# **Space X  Falcon 9 First Stage Landing Prediction**


 ## part 2: Data wrangling (SQL)


Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This objective requires the loading of the spacex dataset.

the data is in the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



In [1]:
!pip install sqlalchemy==1.3.9


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.9 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=70d0a03ccf469d4c634fe5caea5c77b99c7edd764cd36d33c92fe315ab59e022
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [4]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [5]:
!pip install -q pandas==1.1.5

In [6]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [7]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  both result in 0.1234 being formatted as 0.12.


**Note:**
1) the above is a futuer warning
2) This below code is added to remove blank rows from table


In [8]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

## objectives

### objective 1

##### Displaying the names of the unique launch sites  in the space mission


In [23]:
%%sql
select distinct Launch_Site from SPACEXTABLE 

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40



### objective 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [33]:
%%sql
select * from SPACEXTABLE where Launch_Site like 'CCA%'
limit 5


 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-04-06,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-08-12,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-08-10,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-01-03,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### objective 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [39]:
%%sql
select sum(PAYLOAD_MASS__KG_) from SPACEXTABLE 
where Customer = 'NASA (CRS)'

 * sqlite:///my_data1.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


### objective 4




##### Display average payload mass carried by booster version F9 v1.1


In [46]:
%%sql 
Select avg(PAYLOAD_MASS__KG_)
from SPACEXTABLE
where Booster_Version = 'F9 v1.1'

 * sqlite:///my_data1.db
Done.


avg(PAYLOAD_MASS__KG_)
2928.4


### objective 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


In [72]:
%%sql 
select *
from SPACEXTABLE
where (Landing_Outcome = 'Success (ground pad)') AND 
       Date = (select min(Date) 
               from SPACEXTABLE
               where (Landing_Outcome = 'Success (ground pad)'))

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2015-12-22,01:29:00,F9 FT B1019,CCAFS LC-40,OG2 Mission 2 11 Orbcomm-OG2 satellites,2034,LEO,Orbcomm,Success,Success (ground pad)


### objective 6

##### Listing the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [80]:
%%sql
select distinct Booster_Version
from SPACEXTABLE
where Landing_Outcome = 'Success (drone ship)' and (PAYLOAD_MASS__KG_ between 4000 and 6000)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### objective 7
##### Listing the total number of successful and failure mission outcomes

In [95]:
%%sql 
select 
    (select count(*)
     from SPACEXTABLE 
     where Mission_Outcome like '%Success%') as success, 
     (select count(*) 
     from SPACEXTABLE 
     where Mission_Outcome not like '%Success%') as faliuer




 * sqlite:///my_data1.db
Done.


success,faliuer
100,1


### objective 8
##### Listing the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [101]:
%%sql
select distinct Booster_Version 
from SPACEXTABLE
where PAYLOAD_MASS__KG_ = 
    (select max(PAYLOAD_MASS__KG_) from SPACEXTABLE)

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### objective 9
##### Listing the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

In [163]:
%%sql 
select   
    CASE 
    when substr(Date, 6, 2) = '10' then 'October'
    when substr(Date, 6, 2) = '04' then 'April'
    end as monthes, 
    Landing_Outcome, Booster_Version, Launch_Site 
from SPACEXTABLE 
where (substr(Date, 4,1 ) = '5') and (Landing_Outcome = 'Failure (drone ship)')


 * sqlite:///my_data1.db
Done.


monthes,Landing_Outcome,Booster_Version,Launch_Site
October,Failure (drone ship),F9 v1.1 B1012,CCAFS LC-40
April,Failure (drone ship),F9 v1.1 B1015,CCAFS LC-40


### objective 10
##### Ranking the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [192]:
%%sql
select distinct Launch_Site
from SPACEXTABLE

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [190]:
%%sql
select Date, cast(Date as Datetime) as date_object
from SPACEXTABLE
where (date_object <= '2010-06-04') and (date_object >= '2017-03-20')

 * sqlite:///my_data1.db
Done.


Date,date_object
